# Bias Mitigation Experiments - COMPAS

## Table of Contents

<!-- To return to the table of contents, click on the number at any major section heading.

* [1. Prepare COMPAS data](#1.-Prepare-COMPAS-data)

* [2. Training Baseline Models](#2.-Training-Baseline-Models)
    * [2.1 Logistic Regression](#2.1-Learning-a-Logistic-Regression-(LR)-classifier-on-original-data)
    * [2.2 Random Forest](#2.2-Learning-a-Random-Forest-(RF)-classifier-on-original-data)

* [3. Bias mitigation using pre-processing technique](#3.-Bias-mitigation-using-pre-processing-technique)

   * [3.1 Reweighing](#3.1-Reweighing)
       * [3.1.1 RW on Logistic Regression](#3.1.1.-Learning-a-Logistic-Regression-(LR)-classifier-on-data-transformed-by-reweighing)
       * [3.1.2 RW on Random Forest](#3.1.2.-Learning-a-Random-Forest-(RF)-classifier-on-data-transformed-by-reweighing)
   * [3.2 Disparate impact remover ](#3.2-Disparate-impact-remover)
       * [3.2.1 DIR on Logistic Regression](#3.2.1-Learning-a-Logistic-Regression-(LR)-classifier-on-data-transformed-by-DIR)
       * [3.2.2 DIR on Random Forest](#3.2.2-Learning-a-Random-Forest-(RF)-classifier-on-data-transformed-by-DIR)
  
* [4. Bias mitigation using in-processing technique](#4.-Bias-mitigation-using-in-processing-technique)
    * [4.1 Prejudice Remover (PR)](#4.1-Prejudice-Remover-(PR))
    * [4.2 Adversarial Debiasing](#4.2-Adversarial-Debiasing)

* [5. Bias mitigation using post-processing technique](#5.-Bias-mitigation-using-post-processing-technique)
    * [5.1 Reject option classification (ROC)](#5.1-Reject-option-classification-(RejOpCl))
        * [5.1.1 ROC on Logistic Regression](#5.1.1-Reject-option-classification-on-Logistic-Regression)
        * [5.1.2 ROC on Random Forest](#5.1.2-Reject-option-classification-on-Random-Forest)
    * [5.2 Equalized odds processor (EOPP)](#5.2-Equalized-odds-processor-(EqOddsPr))
        * [5.2.1 EOPP on Logistic Regression](#5.2.1-Equalized-odds-processor-on-Logistic-Regression)
        * [5.2.2 EOPP on Random Forest](#5.2.2-Equalized-odds-processor-on-Random-Forest)

* [6. Summary of results](#6.-Summary-of-Model-Learning-Results)

* [7. Analyze rates of different mitigators](#7.-Analyze-rates-of-different-mitigators)

 -->

In [1]:
# to ignore warnings
import warnings
# warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
tf.disable_eager_execution() # comment out when running TabTransformer models

2023-02-04 14:28:23.546698: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-04 14:28:23.546737: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## [1.](#Table-of-Contents) Prepare COMPAS data

In [3]:
# read in data as pandas data frame
df = pd.read_csv("../data/Compas/compas-scores-two-years.csv")
# check columns
pd.set_option('display.max_columns', None) # expand all columns
df.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_jail_in,c_jail_out,c_case_number,c_offense_date,c_arrest_date,c_days_from_compas,c_charge_degree,c_charge_desc,is_recid,r_case_number,r_charge_degree,r_days_from_arrest,r_offense_date,r_charge_desc,r_jail_in,r_jail_out,violent_recid,is_violent_recid,vr_case_number,vr_charge_degree,vr_offense_date,vr_charge_desc,type_of_assessment,decile_score.1,score_text,screening_date,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,0,1,0,0,0,-1.0,2013-08-13 06:03:42,2013-08-14 05:41:20,13011352CF10A,2013-08-13,NaN,1.0,F,Aggravated Assault w/Firearm,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,1,Low,2013-08-14,Risk of Violence,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,0,3,0,0,0,-1.0,2013-01-26 03:45:27,2013-02-05 05:36:53,13001275CF10A,2013-01-26,NaN,1.0,F,Felony Battery w/Prior Convict,1,13009779CF10A,(F3),NaN,2013-07-05,Felony Battery (Dom Strang),NaN,NaN,NaN,1,13009779CF10A,(F3),2013-07-05,Felony Battery (Dom Strang),Risk of Recidivism,3,Low,2013-01-27,Risk of Violence,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,0,4,0,1,4,-1.0,2013-04-13 04:58:34,2013-04-14 07:02:04,13005330CF10A,2013-04-13,NaN,1.0,F,Possession of Cocaine,1,13011511MM10A,(M1),0.0,2013-06-16,Driving Under The Influence,2013-06-16,2013-06-16,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,4,Low,2013-04-14,Risk of Violence,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,0,8,1,0,1,NaN,NaN,NaN,13000570CF10A,2013-01-12,NaN,1.0,F,Possession of Cannabis,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,8,High,2013-01-13,Risk of Violence,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
4,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,0,1,0,0,2,NaN,NaN,NaN,12014130CF10A,NaN,2013-01-09,76.0,F,arrest case no charge,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,1,Low,2013-03-26,Risk of Violence,1,Low,2013-03-26,NaN,NaN,2,0,1102,0,0


In [4]:
# select features of interest
df = df[['sex','age','race','juv_fel_count','juv_misd_count','juv_other_count','priors_count','c_charge_degree', 'two_year_recid']]

# unique values of the protected feature race
df['race'].value_counts()

African-American    3696
Caucasian           2454
Hispanic             637
Other                377
Asian                 32
Native American       18
Name: race, dtype: int64

In [5]:
# encode protected features 

# make race (=protected attribute) a binary variable
array = ['Caucasian', 'African-American'] # only include samples corresponding to Caucasian or African-American
df = df.loc[df['race'].isin(array)]

def protected_race(row):
    if row['race'] == 'Caucasian': # Caucasian is the privileged group in this case
        value = 1
    else:
        value = 0
    return value

df['race'] = df.apply(protected_race, axis=1)

# make sex a binary variable
def protected_sex(row):
    if row['sex'] == 'Female': # female is the privileged group in this case
        value = 1
    else:
        value = 0
    return value

df['sex'] = df.apply(protected_sex, axis=1)

In [6]:
# # one hot encoding of object features
# categorical_cols = list(df.select_dtypes(exclude=[np.number]).columns)
# df = pd.get_dummies(df, columns = categorical_cols)

# # encode binary feature: c_charge 
# df = df.drop(['c_charge_degree_F'], axis=1)
# reset index
df = df.reset_index(drop=True)

In [7]:
# flip labels to ensure pos_label = 1, not becoming recidivist
df['two_year_recid'] = (~df['two_year_recid'].astype(bool)).astype(int)

# select descriptive features and target variable
X = df.loc[:, df.columns != 'two_year_recid'] # select all features but target feature
y = df[['race', 'two_year_recid']] # include protected feature in order to AIF360

In [8]:
# set protected attribute as index
X = X.set_index(['race'], append = True, drop = False)
y = y.set_index(['race'], append = True)

# make y data frames to 1d array to pass modeling, but keep index (protected attribute)
y = pd.Series(y['two_year_recid'], index=y.index)

In [9]:
pos_label = 1
priv_group = 1

In [10]:
X.to_pickle('../data/Compas/COMPAS_X')
y.to_pickle('../data/Compas/COMPAS_y')

## [2.](#Table-of-Contents) Proposed GridSearch Approach 

The GridSearch Approach includes hyperparameter, threshold, and Bias Mitigation

In [11]:
# from FairGridSearch import *

In [12]:
from FairGridSearch import *
%load_ext autoreload
%autoreload 2
# allow automatic reloading of changes in FairGridSearch file

pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[FairAdapt]'


In [13]:
# None, better
# 'RW', worse
# 'LFR_pre', stuck
# 'LFR_in', stuck
# 'AD', stuck
# 'EGR',better
# 'ROC', better
# 'CEO', better

## 3. Case Study: COMPAS

In [14]:
X = pd.read_pickle('../data/Compas/COMPAS_X')
y = pd.read_pickle('../data/Compas/COMPAS_y')

In [15]:
# define desired metric for the use case
ACC_METRIC = 'avg_norm_mcc_score'
FAIR_METRIC = 'abs_avg_ppvd_score'

### Logistic Regression

In [16]:
# model for Reweighing
import timeit
start = timeit.default_timer()

param_grid = {'hyperp_grid': {'C':[1, 10],'solver':['liblinear', 'saga'],'penalty':['l2']}, 
              'threshold': np.linspace(0.3, 0.7, 5),
              'Bias_Mitigation':[None,'RW','LFR_pre','LFR_in','AD','EGR','ROC','CEO']}

clf_lr = fair_GridsearchCV(base='LR', param_grid=param_grid, pos_label=pos_label, priv_group=priv_group,
                           cv=10, n_jobs=multiprocessing.cpu_count()-1)
if __name__ == '__main__':
    clf_lr.fit(X=X, y=y)
    
results_lr = clf_lr.output_table
print(clf_lr._best_param)

stop = timeit.default_timer()
runtime = stop - start
if runtime < 60:
    print('Time: ', runtime, 'sec')
else: print('Time: ', runtime/60, 'min')
results_lr.to_pickle('./Compas_results/Compas_results_LR')
style_table(results_lr)

  0%|          | 0/4 [00:00<?, ?it/s]

{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: False Omission Rate is ill-defined and being set to 0.0 due to no predicted negative samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: False Omission Rate is ill-defined and being set to 0.0 due to no predicted negative samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: False Omission Rate is ill-defined and being set to 0.0 due to no predicted negative samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

{'C': 1, 'penalty': 'l2', 'solver': 'saga'}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: False Omission Rate is ill-defined and being set to 0.0 due to no predicted negative samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: False Omission Rate is ill-defined and being set to 0.0 due to no predicted negative samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: False Omission Rate is ill-defined and being set to 0.0 due to no predicted negative samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

...

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: False Omission Rate is ill-defined and being set to 0.0 due to no predicted negative samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: False Omission Rate is ill-defined and being set to 0.0 due to no predicted negative samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: False Omission Rate is ill-defined and being set to 0.0 due to no predicted negative samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

{'C': 10, 'penalty': 'l2', 'solver': 'saga'}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: False Omission Rate is ill-defined and being set to 0.0 due to no predicted negative samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/studio-lab-user/.conda/envs/default/lib

base_estimator                                                    LR
param              {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
Bias_Mitigation                                                  CEO
threshold                                                        0.5
Name: 117, dtype: object
Time:  9.023075232249994 min


,base_estimator,param,Bias_Mitigation,threshold,avg_acc_score,avg_bacc_score,avg_f1_score,avg_auc_score,avg_mcc_score,avg_norm_mcc_score,avg_spd_score,avg_aod_score,avg_eod_score,avg_ford_score,avg_ppvd_score,avg_(1-consistency_score),avg_gei_score,avg_ti_score,cost
0,LR,"{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}",None,0.300000,0.599675,0.574001,0.717826,0.722252,0.230838,0.615419,-0.111578,-0.101662,-0.047617,-0.040474,-0.085145,0.041789,0.075919,0.084654,0.273003
1,LR,"{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}",None,0.400000,0.644878,0.625993,0.731284,0.722252,0.307816,0.653908,-0.167803,-0.145699,-0.090881,-0.005364,-0.069432,0.055089,0.092411,0.110363,0.178289
2,LR,"{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}",None,0.500000,0.674797,0.668928,0.712626,0.722252,0.344032,0.672016,-0.259550,-0.227603,-0.177167,-0.052065,-0.022598,0.054667,0.141747,0.191353,0.068434
3,LR,"{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}",None,0.600000,0.631057,0.641059,0.587770,0.722252,0.292908,0.646454,-0.272406,-0.240094,-0.251880,-0.067096,-0.000951,0.053496,0.247697,0.353812,0.081140
4,LR,"{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}",None,0.700000,0.571545,0.592756,0.409891,0.722252,0.234684,0.617342,-0.204943,-0.179076,-0.232693,-0.074657,-0.017874,0.038504,0.359579,0.509255,0.177714
5,LR,"{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}",RW,0.300000,0.595610,0.569373,0.716542,0.710707,0.223255,0.611627,-0.035465,-0.018977,-0.017801,-0.022973,-0.116823,0.044520,0.074609,0.082661,0.352908
6,LR,"{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}",RW,0.400000,0.628943,0.609497,0.720747,0.710707,0.270825,0.635412,-0.019245,0.007191,0.010330,-0.115990,-0.113930,0.056390,0.095209,0.115446,0.345343
7,LR,"{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}",RW,0.500000,0.653171,0.646175,0.697547,0.710707,0.299696,0.649848,0.044163,0.077048,0.108109,-0.187841,-0.098558,0.062341,0.146172,0.197301,0.394315
8,LR,"{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}",RW,0.600000,0.631545,0.641380,0.589259,0.710707,0.293352,0.646676,-0.017352,0.012480,0.050168,-0.151855,-0.055709,0.049593,0.246495,0.352042,0.335972
9,LR,"{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}",RW,0.700000,0.565203,0.587241,0.390756,0.710707,0.226524,0.613262,-0.056577,-0.035529,-0.040883,-0.116220,-0.052975,0.037691,0.370796,0.523575,0.330161


### Random Forest

In [17]:
import timeit
start = timeit.default_timer()

param_grid = {'hyperp_grid': {'n_estimators':[10, 100],'criterion':['gini', 'entropy'], 'max_depth':[16]}, 
              'threshold': np.linspace(0.3, 0.7, 5),
              'Bias_Mitigation':[None,'RW','LFR_pre','EGR','ROC','CEO']}

clf_rf = fair_GridsearchCV(base='RF', param_grid=param_grid, pos_label=pos_label, priv_group=priv_group,
                           cv=10, n_jobs=multiprocessing.cpu_count()-1)
if __name__ == '__main__':
    clf_rf.fit(X=X, y=y)
    
results_rf = clf_rf.output_table
print(clf_rf._best_param)

stop = timeit.default_timer()
runtime = stop - start
if runtime < 60:
    print('Time: ', runtime, 'sec')
else: print('Time: ', runtime/60, 'min')
results_rf.to_pickle('./Compas_results/Compas_results_RF')
style_table(results_rf)

  0%|          | 0/4 [00:00<?, ?it/s]

{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and be

{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 100}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: False Omission Rate is ill-defined and being set to 0.0 due to no predicted negative samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: False Omission Rate is ill-defined and being set to 0.0 due to no predicted negative samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

...

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use grou

{'criterion': 'entropy', 'max_depth': 16, 'n_estimators': 100}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: False Omission Rate is ill-defined and being set to 0.0 due to no predicted negative samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: False Omission Rate is ill-defined and being set to 0.0 due to no predicted negative samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

base_estimator                                                    RF
param              {'criterion': 'gini', 'max_depth': 16, 'n_esti...
Bias_Mitigation                                                  CEO
threshold                                                        0.5
Name: 57, dtype: object
Time:  8.88530981453335 min


,base_estimator,param,Bias_Mitigation,threshold,avg_acc_score,avg_bacc_score,avg_f1_score,avg_auc_score,avg_mcc_score,avg_norm_mcc_score,avg_spd_score,avg_aod_score,avg_eod_score,avg_ford_score,avg_ppvd_score,avg_(1-consistency_score),avg_gei_score,avg_ti_score,cost
0,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",None,0.300000,0.615610,0.598383,0.703140,0.663190,0.230911,0.615455,-0.106297,-0.089824,-0.037734,-0.155259,-0.077496,0.183870,0.110417,0.140535,0.278247
1,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",None,0.400000,0.627317,0.616621,0.689357,0.663190,0.246955,0.623477,-0.119614,-0.097943,-0.050725,-0.140409,-0.069217,0.209138,0.138534,0.184877,0.256908
2,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",None,0.500000,0.624390,0.621109,0.655208,0.663190,0.243601,0.621800,-0.163223,-0.142134,-0.087508,-0.141510,-0.041868,0.231089,0.179442,0.247596,0.214976
3,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",None,0.600000,0.610244,0.614493,0.601527,0.663190,0.229984,0.614992,-0.171675,-0.150868,-0.106398,-0.134833,-0.024559,0.240195,0.230142,0.322105,0.213333
4,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",None,0.700000,0.578699,0.590770,0.510974,0.663190,0.193138,0.596569,-0.154254,-0.137963,-0.103684,-0.134942,-0.008952,0.230504,0.300365,0.418587,0.249177
5,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",RW,0.300000,0.610732,0.593396,0.699751,0.660284,0.219199,0.609599,-0.062055,-0.045177,0.000197,-0.175159,-0.084772,0.183480,0.111536,0.142540,0.328346
6,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",RW,0.400000,0.620650,0.609560,0.685165,0.660284,0.233071,0.616535,-0.074983,-0.055433,-0.000400,-0.175014,-0.069537,0.212000,0.139298,0.186063,0.308482
7,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",RW,0.500000,0.619187,0.615167,0.653931,0.660284,0.232207,0.616104,-0.089016,-0.069172,-0.009063,-0.169990,-0.051353,0.231740,0.177888,0.245035,0.294880
8,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",RW,0.600000,0.608943,0.613295,0.599541,0.660284,0.227649,0.613824,-0.096230,-0.076152,-0.020008,-0.158787,-0.030064,0.244715,0.231567,0.324046,0.289946
9,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",RW,0.700000,0.585691,0.597399,0.522257,0.660284,0.206279,0.603139,-0.091651,-0.074004,-0.029989,-0.147578,-0.016628,0.232065,0.292466,0.408987,0.305210


### Gradient Boosting

In [18]:
import timeit
start = timeit.default_timer()

param_grid = {'hyperp_grid': {'n_estimators':[10, 100],'criterion':['friedman_mse'],'max_depth':[8, 32]}, 
              'threshold': np.linspace(0.3, 0.7, 5),
              'Bias_Mitigation':[None,'RW','LFR_pre','EGR','ROC','CEO']}

clf_gb = fair_GridsearchCV(base='GB', param_grid=param_grid, pos_label=pos_label, priv_group=priv_group,
                           cv=10, n_jobs=multiprocessing.cpu_count()-1)
if __name__ == '__main__':
    clf_gb.fit(X=X, y=y)
    
results_gb = clf_gb.output_table
print(clf_gb._best_param)

stop = timeit.default_timer()
if runtime < 60:
    print('Time: ', runtime, 'sec')
else: print('Time: ', runtime/60, 'min')
results_gb.to_pickle('./Compas_results/Compas_results_GB')
style_table(results_gb)

  0%|          | 0/4 [00:00<?, ?it/s]

{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: False Omission Rate is ill-defined and being set to 0.0 due to no predicted negative samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_e

{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 100}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and be

...

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in D

{'criterion': 'friedman_mse', 'max_depth': 32, 'n_estimators': 100}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in D

base_estimator                                                    GB
param              {'criterion': 'friedman_mse', 'max_depth': 8, ...
Bias_Mitigation                                                  CEO
threshold                                                        0.5
Name: 27, dtype: object
Time:  8.88530981453335 min


,base_estimator,param,Bias_Mitigation,threshold,avg_acc_score,avg_bacc_score,avg_f1_score,avg_auc_score,avg_mcc_score,avg_norm_mcc_score,avg_spd_score,avg_aod_score,avg_eod_score,avg_ford_score,avg_ppvd_score,avg_(1-consistency_score),avg_gei_score,avg_ti_score,cost
0,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",None,0.300000,0.585691,0.558000,0.713716,0.709030,0.205935,0.602967,-0.078145,-0.070057,-0.027467,-0.084818,-0.094617,0.076846,0.071011,0.077126,0.318888
1,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",None,0.400000,0.639024,0.620677,0.725053,0.709030,0.290908,0.645454,-0.164981,-0.145556,-0.078278,-0.081382,-0.062511,0.112228,0.096960,0.118003,0.189565
2,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",None,0.500000,0.660650,0.653442,0.705004,0.709030,0.315194,0.657597,-0.214521,-0.185615,-0.133958,-0.084464,-0.038121,0.125528,0.141737,0.190566,0.127882
3,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",None,0.600000,0.635447,0.643286,0.606692,0.709030,0.292767,0.646383,-0.151591,-0.119631,-0.110287,-0.104059,-0.038681,0.126081,0.233329,0.331968,0.202026
4,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",None,0.700000,0.534634,0.559683,0.302317,0.709030,0.174600,0.587300,-0.125080,-0.109205,-0.138647,-0.100275,-0.044276,0.093041,0.424384,0.586527,0.287620
5,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",RW,0.300000,0.586504,0.558784,0.714414,0.698300,0.207888,0.603944,-0.045359,-0.034228,-0.014248,-0.074723,-0.107467,0.078764,0.070618,0.076438,0.350697
6,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",RW,0.400000,0.633984,0.615036,0.723008,0.698300,0.281626,0.640813,-0.084238,-0.062670,-0.015362,-0.141592,-0.081647,0.115545,0.095886,0.116354,0.274949
7,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",RW,0.500000,0.653984,0.646434,0.700520,0.698300,0.301329,0.650665,-0.069164,-0.038242,-0.003989,-0.140962,-0.077299,0.136911,0.142928,0.192255,0.280171
8,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",RW,0.600000,0.631870,0.639058,0.607064,0.698300,0.283026,0.641513,-0.000101,0.030090,0.060512,-0.150672,-0.077696,0.122081,0.232145,0.329231,0.358386
9,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",RW,0.700000,0.524715,0.550549,0.274185,0.698300,0.154687,0.577343,-0.005671,0.004980,0.020629,-0.130648,-0.050535,0.088650,0.441825,0.605819,0.416986


### Support Vector Machine

In [ ]:
import timeit
start = timeit.default_timer()

param_grid = {'hyperp_grid': {'kernel':['rbf','linear','poly','sigmoid'],'gamma':['scale']},
              'threshold': np.linspace(0.3, 0.7, 5),
              'Bias_Mitigation':[None,'RW','LFR_pre','EGR','ROC','CEO']}

clf_svm = fair_GridsearchCV(base='SVM', param_grid=param_grid, pos_label=pos_label, priv_group=priv_group,
                           cv=10, n_jobs=multiprocessing.cpu_count()-1)
if __name__ == '__main__':
    clf_svm.fit(X=X, y=y)
    
results_svm = clf_svm.output_table
print(clf_svm._best_param)

stop = timeit.default_timer()
runtime = stop - start
if runtime < 60:
    print('Time: ', runtime, 'sec')
else: print('Time: ', runtime/60, 'min')
results_svm.to_pickle('./Compas_results/Compas_results_SVM')
style_table(results_svm)

  0%|          | 0/4 [00:00<?, ?it/s]

{'gamma': 'scale', 'kernel': 'rbf'}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/studio-lab-user/.conda/envs/def

{'gamma': 'scale', 'kernel': 'linear'}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in D

### Gaussian Naive Bayes

In [ ]:
# model for Reweighing
import timeit
start = timeit.default_timer()

param_grid = {'hyperp_grid': {'var_smoothing': np.logspace(0,-9, num=4)}, 
              'threshold': np.linspace(0.3, 0.7, 5),
              'Bias_Mitigation':[None,'RW','LFR_pre','EGR','ROC','CEO']}

clf_nb = fair_GridsearchCV(base='NB', param_grid=param_grid, pos_label=pos_label, priv_group=priv_group,
                           cv=10, n_jobs=multiprocessing.cpu_count()-1)
if __name__ == '__main__':
    clf_nb.fit(X=X, y=y)
    
results_nb = clf_nb.output_table
# print(clf._best_param)

stop = timeit.default_timer()
runtime = stop - start
if runtime < 60:
    print('Time: ', runtime, 'sec')
else: print('Time: ', runtime/60, 'min')
results_nb.to_pickle('./Compas_results/Compas_results_NB')
style_table(results_nb)

### TabTransformer

In [15]:
import timeit
start = timeit.default_timer()

param_grid = {'hyperp_grid': {'epochs':[20, 30],'learing_rate':[1e-04, 1e-05]},
              'threshold': np.linspace(0.3, 0.7, 5),
              'Bias_Mitigation':[None,'RW','ROC','CEO']}
# param_grid = {'hyperp_grid': {'epochs':[3]}, 'threshold':[0.5],
#               'Bias_Mitigation':[None,'RW','ROC','CEO']}

clf_tab = fair_GridsearchCV(base='TabTrans',param_grid=param_grid, pos_label=pos_label, priv_group=priv_group,
                           cv=10, n_jobs=multiprocessing.cpu_count()-1)

clf_tab.fit(X=X, y=y)
results_tab = clf_tab.output_table
# print(clf._best_param)

stop = timeit.default_timer()
runtime = stop - start
if runtime < 60:
    print('Time: ', runtime, 'sec')
else: print('Time: ', runtime/60, 'min')
results_tab.to_pickle('./Compas_results/Compas_results_TabTrans')
style_table(results_tab)

  0%|          | 0/4 [00:00<?, ?it/s]

{'epochs': 20, 'learing_rate': 0.0001}



100%|██████████| 1/1 [00:00<00:00, 34.98it/s]/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/tabtransformertf/utils/preprocessing.py:21: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  dataset[key] = value[:, tf.newaxis]

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/tabtransformertf/utils/preprocessing.py:21: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  dataset[key] = value[:, tf.newaxis]
100%|██████████| 1/1 [00:00<00:00, 33.11it/s]
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/tabtransformertf/utils/preprocessing.py:21: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future ver

------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


  0%|          | 0/1 [00:00<?, ?it/s]/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/tabtransformertf/utils/preprocessing.py:27: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  dataset[key] = value[:, tf.newaxis]
100%|██████████| 1/1 [00:00<00:00, 300.17it/s]/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/tabtransformertf/utils/preprocessing.py:27: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  dataset[key] = value[:, tf.newaxis]


100%|██████████| 1/1 [00:00<00:00, 198.54it/s]


...

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
9/9 [==============================] - 10s 360ms/step - loss: 0.6933 - PR AUC: 0.6211 - val_loss: 0.6500 - val_PR AUC: 0.6451
Epoch 2/30
9/9 [==============================] - 1s 72ms/step - loss: 0.6862 - PR AUC: 0.6264 - val_loss: 0.6499 - val_PR AUC: 0.6457
Epoch 3/30
9/9 [==============================] - 1s 86ms/step - loss: 0.6943 - PR AUC: 0.6165 - val_loss: 0.6497 - val_PR AUC: 0.6452
Epoch 4/30
9/9 [==============================] - 1s 71ms/step - loss: 0.6908 - PR AUC: 0.6269 - val_loss: 0.6497 - val_PR AUC: 0.6452
Epoch 5/30
9/9 [==============================] - 1s 69ms/step - loss: 0.6924 - PR AUC: 0.6186 - val_loss: 0.6496 - val_PR AUC: 0.6444
Epoch 6/30
9/9 [==============================] - 1s 81ms/step - loss: 0.6874 - PR AUC: 0.6184 - val_loss: 0.6495 - val_PR AUC: 0.6452
Epoch 7/30
9/9 [==============================] - 1s 91ms/step - loss: 0.6944 - PR AUC: 0.6083 - val_loss: 0.6495 - val_PR AUC: 0.6447
Epoch 8/30
9/9 [==============================] - 1s 

100%|██████████| 4/4 [32:34<00:00, 488.63s/it]


Time:  32.581826755866665 min


,base_estimator,param,Bias_Mitigation,threshold,avg_acc_score,avg_bacc_score,avg_f1_score,avg_auc_score,avg_mcc_score,avg_norm_mcc_score,avg_spd_score,avg_aod_score,avg_eod_score,avg_ford_score,avg_ppvd_score,avg_(1-consistency_score),avg_gei_score,avg_ti_score,cost
0,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",None,0.300000,0.551707,0.519860,0.702397,0.673135,0.102547,0.551273,-0.031476,-0.028796,-0.014992,0.098283,-0.112748,0.012715,0.061905,0.063786,0.417251
1,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",None,0.400000,0.590081,0.567030,0.700627,0.673135,0.194819,0.597409,-0.113081,-0.102794,-0.069921,-0.026106,-0.093393,0.042537,0.092873,0.111698,0.289509
2,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",None,0.500000,0.578537,0.581747,0.514358,0.673135,0.190526,0.595263,-0.153280,-0.136529,-0.136351,-0.064978,-0.070456,0.060325,0.264917,0.357653,0.251457
3,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",None,0.600000,0.514959,0.537599,0.230108,0.673135,0.083352,0.541676,-0.069191,-0.061873,-0.058710,-0.099635,-0.031629,0.031220,0.448626,0.602020,0.389133
4,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",None,0.700000,0.477073,0.508880,0.048627,0.673135,0.019437,0.509719,-0.019661,-0.016953,-0.021658,-0.115932,-0.011605,0.006407,0.546955,0.731074,0.470620
5,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",RW,0.300000,0.570244,0.541302,0.706800,0.683674,0.155368,0.577684,-0.066386,-0.060994,-0.032576,0.023285,-0.102930,0.022374,0.069577,0.075642,0.355930
6,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",RW,0.400000,0.624553,0.603781,0.721491,0.683674,0.267018,0.633509,-0.157100,-0.139910,-0.091388,-0.000746,-0.079127,0.050309,0.090508,0.107827,0.209391
7,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",RW,0.500000,0.650732,0.641777,0.702677,0.683674,0.295757,0.647879,-0.219904,-0.191067,-0.165404,-0.031320,-0.064065,0.065008,0.137539,0.183553,0.132218
8,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",RW,0.600000,0.569593,0.584804,0.460930,0.683674,0.191736,0.595868,-0.185536,-0.165537,-0.178635,-0.095726,-0.024514,0.074667,0.330452,0.458842,0.218596
9,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",RW,0.700000,0.477398,0.509620,0.054101,0.683674,0.039281,0.519640,-0.026052,-0.023230,-0.027765,-0.117462,-0.307646,0.011870,0.548051,0.732750,0.454308


## 4. Analysis

In [ ]:
# results_dict = {}
# results_dict['results_LR'] = results_LR
# results_dict['results_RF'] = results_RF
# results_dict['results_GB'] = results_GB
# results_dict['results_SVM'] = results_SVM
# results_dict['results_NB'] = results_NB

import os
results_all = pd.DataFrame()
for file in os.listdir(dir_path):
    print(file)
    result = read_pickle('./Compas_results/{}'.format(file))
    results_all = pd.concat([results_all, result])

In [ ]:
results_all

In [ ]:
import seaborn as sns
accuracy_metrics = ['acc_score', 'bacc_score', 'f1_score', 'auc_score', 'mcc_score','norm_mcc_score']
group_fairness = ['spd_score', 'aod_score', 'eod_score','ford_score','ppvd_score']
individual_fairness = ['consistency_score','gei_score','ti_score']
fairness_metrics = group_fairness+individual_fairness
avg_accuracy_metrics = ['avg_'+col for col in accuracy_metrics]
avg_fairness_metrics = ['avg_'+col for col in fairness_metrics]

In [ ]:
results_all_converted = results_all.copy(deep=True)
for col in avg_fairness_metrics:
    abs_col_name = 'abs_'+col
    results_all_converted[abs_col_name] = results_all_converted[col].abs()
results_all_converted.head()

In [ ]:
results_all_converted.base_estimator.value_counts()

In [ ]:
results_all_converted[(results_all_converted.Bias_Mitigation=='LFR_in')&(results_all_converted.threshold==0.5)]

### 4.1 Accuracy vs. Fairness

#### 4.1.1 General Results

We can see from the graphs below:
1. Overall trend: the higher the accuracy, the lower the group fairness (positive slope)
2. But the choice of metrics does matter: 
    - group fairness: ford_score showcased no "trade-offs"
    - accuracy metrics: f1_score and roc_auc_score showed milder "trade-offs"
    - same fairness metric combined with different accuracy metrics can present both positive/negative relations, vice versa

In [ ]:
abs_fairness_metrics = ['abs_'+col for col in avg_fairness_metrics]
g = sns.pairplot(data=results_all_converted, x_vars=avg_accuracy_metrics, y_vars=abs_fairness_metrics,
             height=5, aspect=.8, kind="reg")
g.map(sns.scatterplot)

#### 4.1.2 Base_estimator specific Results

We can see from the graphs below:
1. Different base_estimators can also generate very different results: 
    - SVM (red) the least robust, while "NB" manifest clear trend
    - base_estimators divided into two groups when it comes to consistency_score: tree-based, ensemble (trade-offs) vs non tree-based, non ensemble

In [ ]:
g = sns.lmplot(data=results_all_converted, x=ACC_METRIC, y=FAIR_METRIC, hue='base_estimator', height=4)

In [ ]:
g = sns.pairplot(data=results_all_converted[results_all_converted.base_estimator!=''],
                 x_vars=avg_accuracy_metrics, y_vars=abs_fairness_metrics,
                 height=5, aspect=.8, kind="reg", hue='base_estimator')
g.map(sns.scatterplot)

In [ ]:
abs_fairness_metrics = ['abs_'+col for col in avg_fairness_metrics]
for base in results_all_converted.base_estimator.unique():
    print(base)
    if base=='':
        pass
    else:
        g = sns.pairplot(data=results_all_converted[results_all_converted.base_estimator==base],
                         x_vars=avg_accuracy_metrics, y_vars=abs_fairness_metrics,
                         height=5, aspect=.8, kind="reg")
        g.map(sns.scatterplot)
        plt.show()

#### 4.1.3 Threshold specific Results

We can see from the graphs below:
1. Using different threshold also yiels different results: 
    - most obvious with (f1_score, group_fairness) combinations

In [ ]:
g = sns.lmplot(data=results_all_converted, x=ACC_METRIC, y=FAIR_METRIC, hue='threshold', height=4)

In [ ]:
for base in results_all_converted_try.base_estimator.unique():
    g = sns.lmplot(data=results_all_converted[results_all_converted.base_estimator==base],
                   x="avg_norm_mcc_score", y="abs_avg_ppvd_score", hue="threshold", height=3).fig.suptitle(base)

In [ ]:
g = sns.pairplot(data=results_all_converted,
                 x_vars=avg_accuracy_metrics, y_vars=abs_fairness_metrics,
                 height=5, aspect=.8, kind="reg", hue='threshold')
g.map(sns.scatterplot)

#### 4.1.4 Bias_mitigation specific Results

We can see from the graphs below:
1. Results are clearly divided into two groups
2. Generally with wide C.I.

In [ ]:
results_all_converted['BM_Category'] = results_all_converted['Bias_Mitigation'].replace({v: k for k, l in bm_category.items()\
                                                                                         for v in l})
results_all_converted[['Bias_Mitigation','BM_Category']].drop_duplicates()

In [ ]:
g = sns.pairplot(data=results_all_converted[results_all_converted.BM_Category!='None'],
                 x_vars=avg_accuracy_metrics, y_vars=abs_fairness_metrics,
                 height=5, aspect=.8, kind="reg", hue='Bias_Mitigation', plot_kws={'scatter_kws': {'s': 2}})
g.map(sns.scatterplot)

In [ ]:
# results_all_converted_try = results_all_converted.copy(deep=True).reset_index(drop=True)
# for i in results_all_converted_try.index:
#     if results_all_converted_try.loc[i, 'base_estimator'] == '':
#         results_all_converted_try.loc[i, 'base_estimator'] = results_all_converted_try.loc[i-1, 'base_estimator']
        
# results_all_converted_try.reset_index(drop=True).iloc[-30:,]

In [ ]:
for base in results_all_converted_try.base_estimator.unique():
    g = sns.lmplot(data=results_all_converted_try[results_all_converted_try.base_estimator==base],
                   x="avg_norm_mcc_score", y="abs_avg_ppvd_score", hue="Bias_Mitigation", height=3).fig.suptitle(base)

##### Bias Mitigation Category

Pre- and post-processing always generate similar results, while in-processing is usually different:
    - less trade-offs between group fairness and accuracy
    - but trade-offs for "ford_score", which is the very g_fair metric showing no trade-offs in overall results

In [ ]:
for base in results_all_converted_try.base_estimator.unique():
    g = sns.lmplot(data=results_all_converted[results_all_converted.base_estimator==base],
                   x="avg_norm_mcc_score", y="abs_avg_ppvd_score", hue="BM_Category", height=3).fig.suptitle(base)

In [ ]:
g = sns.pairplot(data=results_all_converted[results_all_converted.BM_Category!='None'],
                 x_vars=avg_accuracy_metrics, y_vars=abs_fairness_metrics,
                 height=5, aspect=.8, kind="reg", hue='BM_Category', plot_kws={'scatter_kws': {'s': 2}})
g.map(sns.scatterplot)

### 4.2 Group vs. Individual Fairness

No clear trend between group and individual fairness with a few exceptions

In [ ]:
g = sns.pairplot(data=results_all_converted,
                 x_vars=['abs_avg_'+col for col in group_fairness], y_vars=['abs_avg_'+col for col in individual_fairness],
             height=5, aspect=.8, kind="reg")
g.map(sns.scatterplot)

#### 4.2.1 Heatmap

In [ ]:
# avg_accuracy_metrics = ['avg_'+col for col in accuracy_metrics]
# avg_fairness_metrics = ['avg_'+col for col in fairness_metrics]
sns.heatmap(results_all_converted[avg_fairness_metrics].corr(), annot=True, fmt=".1f")

In [ ]:
# avg_accuracy_metrics = ['avg_'+col for col in accuracy_metrics]
# avg_fairness_metrics = ['avg_'+col for col in fairness_metrics]
sns.heatmap(results_all_converted[avg_accuracy_metrics+avg_fairness_metrics].corr(), annot=True, fmt=".1f")